In [66]:
using Plots, LsqFit, TwoFAST, Dierckx, Printf, LaTeXStrings, Test

In [60]:
function derivate_point(xp, yp, x1, y1, x2, y2)
     l2, l1 = (x2 - xp), (xp - x1)
     m2, m1 = (y2 - yp) / l2, (yp - y1) / l1
     res = (m1 * l2 + m2 * l1) / (l1 + l2)
     #println(res)
     return res
end

function derivate_vector(XS, YS; N::Integer = 1)
    @assert length(XS) == length(YS) "xs and ys must have the same length!"
    @assert length(YS) > 2*N "length of xs and ys must be > 2N !"
    
    mean_exp_xs = sum([log10(abs(x)) for x in XS]) / length(XS)
     en_xs = 10.0^(-mean_exp_xs)
     xs = XS .* en_xs

     mean_exp_ys = sum([log10(abs(y)) for y in YS]) / length(YS)
     en_ys = 10.0^(-mean_exp_ys)
     ys = YS .* en_ys
    
     mean_ys = sum(ys)/length(ys)
     @assert !all([isapprox(y/mean_ys, 1.0, rtol=1e-6) for y in ys]) "DO NOT WORK!"
    
    
     if N == 1
          real_vec = [derivate_point(xs[i], ys[i], xs[i-1], ys[i-1], xs[i+1], ys[i+1])
                      for i in (N+1):(length(xs)-N)]
        return vcat(real_vec[begin], real_vec, real_vec[end]) .* (en_xs / en_ys)
     elseif N > 1
          vec = [derivate_point(xs[i], ys[i], xs[i-j], ys[i-j], xs[i+j], ys[i+j])
                 for i in (N+1):(length(xs)-N), j in 1:N]
          real_vec = [sum(row) / N for row in eachrow(vec)]
          return vcat([real_vec[begin] for i in 1:N], real_vec,
            [real_vec[end] for i in 1:N]) .* (en_xs / en_ys)
     else
          throw(ErrorException(" N must be an integer >1, not $N!"))
     end
end

function spectral_index(xs, ys; N::Integer = 1, con = false)
     derivs = derivate_vector(xs, ys; N = N)

     if con == false
          res = [x * d / y for (x, y, d) in zip(xs, ys, derivs)]
        return vcat(
            [res[begin+N] for i in 1:N], 
            res[begin+N:end-N], 
            [res[end-N] for i in 1:N]
        )
     else
          sec_derivs = derivate_vector(xs, derivs; N = N)
          res = [x * d2 / d for (x, d, d2) in zip(xs, derivs, sec_derivs)] .+ 1.0
        return vcat(
            [res[begin+N+1] for i in 1:N+1], 
            res[begin+N+1:end-N-1], 
            [res[end-N-1] for i in 1:N+1]
        )
     end
end

function mean_spectral_index(xs, ys; N::Integer = 1, con = false)
     @assert length(xs) > 2*N + 2 "length of xs and ys must be > 2N+2"
     vec = spectral_index(xs, ys; N = N, con = con)[begin+N+1:end-N-1]
     return sum(vec) / length(vec)
end

mean_spectral_index (generic function with 1 method)

In [56]:
xs = 10 .^ range(-6,-4, length=100)
ys = [3.6e-22 + 3.65*x^(-3.5) for x in xs]
calc = derivate_vector(xs, ys)
[-3.65*3.5*x^(-4.5)/c for (x,c) in zip(xs, calc)]

100-element Vector{Float64}:
 1.221919030616939
 0.9911361984620497
 0.991136198462048
 0.991136198462049
 0.9911361984620497
 0.9911361984620491
 0.9911361984620505
 0.9911361984620476
 0.9911361984620491
 0.9911361984620504
 0.9911361984620476
 0.9911361984620484
 0.9911361984620494
 ⋮
 0.9911361984620497
 0.9911361984620478
 0.9911361984620501
 0.9911361984620498
 0.9911361984620491
 0.9911361984620486
 0.9911361984620493
 0.9911361984620501
 0.9911361984620507
 0.991136198462048
 0.9911361984620493
 0.8039411280842566

In [58]:
xs = 10 .^ range(-6,-4, length=100)
ys = [3.65*x^(-3.5) for x in xs]
calc = spectral_index(xs, ys; N=1, con=false)
[-3.5/c for (x,c) in zip(xs, calc)]

100-element Vector{Float64}:
 0.9911361984620496
 0.9911361984620496
 0.991136198462048
 0.9911361984620487
 0.9911361984620495
 0.9911361984620491
 0.9911361984620504
 0.9911361984620476
 0.9911361984620491
 0.9911361984620505
 0.9911361984620476
 0.9911361984620481
 0.9911361984620494
 ⋮
 0.9911361984620497
 0.9911361984620476
 0.9911361984620503
 0.9911361984620498
 0.991136198462049
 0.9911361984620486
 0.9911361984620491
 0.9911361984620501
 0.9911361984620506
 0.9911361984620479
 0.9911361984620491
 0.9911361984620491

In [64]:
xs = 10 .^ range(-6, -4, length=100)
ys = [3.54e6 * x^(-4.856) for x in xs]
mean_spectral_index(xs, ys; con = false)/(-4.856)

1.0145292451492502

In [67]:
 @test isapprox(mean_spectral_index(xs, ys; con = false), -4.856; atol = 5e-2)

Test Failed at In[67]:1
  Expression: isapprox(mean_spectral_index(xs, ys; con = false), -4.856; atol = 0.05)
   Evaluated: isapprox(-4.9265540144447595, -4.856; atol = 0.05)


LoadError: [91mThere was an error during testing[39m